<a href="https://colab.research.google.com/github/TracieTruong/Visualization-Assignment-2-Redesign-Graphic/blob/main/Redesign_Graphic_Minard_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Redesign a Notable Information Graphic**

# Introduction

In this notebook, I redesigned Charles Minard's map of Napoleon's disastrous Russian campaign of 1812 into an interactive storytelling experience. With this method, it will bring history to life in a captivating way

## Napoleon's Russian campaign of 1812
This was a disastrous military invasion by the French Grande Armée into the Russian Empire, which ultimately led to the army's near-total destruction and a major turning point in the Napoleonic Wars.

The invasion, which started on June 24, 1812, ended in December 1812 and failed because of Russia's scorched-earth policy, the harsh winter, and logistical challenges.

The French emperor, intent on conquering Europe, sent 600,000 troops into Russia. Six disastrous months later, only an estimated 100,000 managed to escape, resulting in a severely weakened Napoleon's empire.

## Charles Minard's map of Napoleon's disastrous Russian campaign of 1812.


<p><img style="float: left;margin:5px 20px 5px 1px" src="https://datavizblog.com/wp-content/uploads/2013/05/minard_09_10_11.jpg"></p>


<p>Charles Joseph Minard is most widely known for a single work, his poignant flow-map depiction of the fate of Napoleon’s Grand Army in the disasterous 1812 Russian campaign. This “Carte figurative des pertes successives en hommes de l’Armee´ Franc¸aise dans la campagne de Russe 1812-1813” has been called “the best graphic ever produced”, one which seemed to “defy the pen of the historian by its brutal eloquence”. (Michael Friendly, A brief biography of Minard, p.1)</p>

<p><img style="float: left;margin:5px 20px 5px 1px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/29/Minard.png/1920px-Minard.png"> </p>


# Iteration 1: Ideas generation
In the beginning, I did some studies about the Napoleon’s Grand Army in the disasterous 1812 Russian campaign and Charles Minard's map as well as Michael Friendly's works.

With the provided datasets, I reworked on Excel as I am more skilled in that. Then I mind-mapped for the ideas
<p><img style="float: left;margin:5px 20px 5px 1px" src="https://media.cmsmax.com/mtasumd2bdfrxqs1knb0i/how-to-visualize-your-data-with-charts-and-graphs-1024x768-1.jpg"> </p>

# Iteration 2: Data preparation and cleaning

To begin, I prepare the notebook environment

In [ ]:
# Import the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Secondly, I import and clean the datasets

In [ ]:
# Import datasets
cities = pd.read_csv('/content/drive/MyDrive/Redesign_Graphic_Minard_Map/minard-cities.csv')
temps = pd.read_csv('/content/drive/MyDrive/Redesign_Graphic_Minard_Map/minard-temp.csv')

I noticed that the dates in "minard-temp" dataset were inaccurate in years, so in the next step, I extract the day and month and combined them with the year of 1812, the original year of the event: Napoleon’s Russian Campaign

In [ ]:
# Clean the dataset
temps = temps.dropna()
temps ["date"] = pd.to_datetime(temps["date"], format="%b%d", errors="coerce")

# Extract day and month from date and create 2 new columns
temps ["day"] = temps ["date"].dt.day
temps ["month"] = temps ["date"].dt.month
temps

,long,temp,days,date,day,month
0,37.6,0,6,1900-10-18,18,10
1,36.0,0,6,1900-10-24,24,10
2,33.2,-9,16,1900-11-09,9,11
3,32.0,-21,5,1900-11-14,14,11
5,28.5,-20,4,1900-11-28,28,11
6,27.2,-24,3,1900-12-01,1,12
7,26.7,-30,5,1900-12-06,6,12
8,25.3,-26,1,1900-12-07,7,12


In [ ]:
# Assign the year to 1812 to all rows
temps['year'] = 1812

# Combine year, month and day by each row into a datetime format and overwrite the old "date" column
temps ["date"] = pd.to_datetime(temps [["year", "month", "day"]].fillna(1).astype(int))

# Display dataframe
temps1 = temps.drop(columns=["year", "month", "day"])
temps1

,long,temp,days,date
0,37.6,0,6,1812-10-18
1,36.0,0,6,1812-10-24
2,33.2,-9,16,1812-11-09
3,32.0,-21,5,1812-11-14
5,28.5,-20,4,1812-11-28
6,27.2,-24,3,1812-12-01
7,26.7,-30,5,1812-12-06
8,25.3,-26,1,1812-12-07


I separate the direction of movement from dataframe cities

In [ ]:
# Create Advance and Reatreat dataframe
advance_cities = cities[cities['direction'] == 'Advance'].reset_index(drop=True)
retreat_cities = cities[cities['direction'] == 'Retreat'].reset_index(drop=True)

The next step, I merge retreat_citites with temps1 dataframe by nearest longitude to have a comparable view of the data.

Due to the lack of information on temperatures and dates in cities, I replace missing values with the data of the nearest longitude.

In [ ]:
# Sort the data
advance_cities_sorted = advance_cities.sort_values("long")
retreat_cities_sorted = retreat_cities.sort_values("long")
temps_sorted = temps1.sort_values("long")

In [ ]:
advance = advance_cities_sorted.dropna().reset_index(drop=True)
advance

,long,lat,survivors,direction,group,city
0,24.0,54.9,340000,Advance,1,Kowno
1,26.0,54.7,320000,Advance,1,Smorgoni
2,28.0,54.9,280000,Advance,1,Gloubokoe
3,28.5,55.0,240000,Advance,1,Polotzk
4,30.3,55.3,175000,Advance,1,Witebsk
5,34.4,55.5,127100,Advance,1,Chjat
6,37.6,55.8,100000,Advance,1,Moscou


In [ ]:
cities_cleaned = cities.dropna().reset_index(drop=True)
cities_cleaned

,long,lat,survivors,direction,group,city
0,24.0,54.9,340000,Advance,1,Kowno
1,26.0,54.7,320000,Advance,1,Smorgoni
2,28.0,54.9,280000,Advance,1,Gloubokoe
3,28.5,55.0,240000,Advance,1,Polotzk
4,30.3,55.3,175000,Advance,1,Witebsk
5,34.4,55.5,127100,Advance,1,Chjat
6,37.6,55.8,100000,Advance,1,Moscou
7,37.7,55.7,100000,Retreat,1,Moscou
8,36.8,55.0,96000,Retreat,1,Mojaisk
9,33.3,54.8,37000,Retreat,1,Dorogobouge


In [ ]:
# Merge cities and temps by nearest longitude
merged_retreat = pd.merge_asof(retreat_cities_sorted, temps_sorted, on="long", direction="nearest")
merged_advance = pd.merge_asof(advance_cities_sorted, temps_sorted, on="long", direction="nearest")

In [ ]:
# Drop missing values and display the dataframe
retreat = merged_retreat.dropna().reset_index(drop=True)
retreat

,long,lat,survivors,direction,group,city,temp,days,date
0,24.4,54.4,4000,Retreat,1,Wilna,-26,1,1812-12-07
1,26.8,54.3,12000,Retreat,1,Moiodexno,-30,5,1812-12-06
2,27.5,54.5,20000,Retreat,1,Minsk,-24,3,1812-12-01
3,28.5,54.2,20000,Retreat,1,Studienska,-20,4,1812-11-28
4,29.2,54.3,20000,Retreat,1,Bobr,-20,4,1812-11-28
5,32.0,54.6,24000,Retreat,1,Smolensk,-21,5,1812-11-14
6,33.3,54.8,37000,Retreat,1,Dorogobouge,-9,16,1812-11-09
7,36.8,55.0,96000,Retreat,1,Mojaisk,0,6,1812-10-24
8,37.7,55.7,100000,Retreat,1,Moscou,0,6,1812-10-18


In [ ]:
# Drop missing values and display the dataframe
advance = merged_advance.dropna().reset_index(drop=True)
advance

,long,lat,survivors,direction,group,city,temp,days,date
0,24.0,54.9,340000,Advance,1,Kowno,-26,1,1812-12-07
1,26.0,54.7,320000,Advance,1,Smorgoni,-26,1,1812-12-07
2,28.0,54.9,280000,Advance,1,Gloubokoe,-20,4,1812-11-28
3,28.5,55.0,240000,Advance,1,Polotzk,-20,4,1812-11-28
4,30.3,55.3,175000,Advance,1,Witebsk,-21,5,1812-11-14
5,34.4,55.5,127100,Advance,1,Chjat,-9,16,1812-11-09
6,37.6,55.8,100000,Advance,1,Moscou,0,6,1812-10-18


# Iteration 3: The output charts

The first map chart is an interactive map that uses scatter_mapbox to visualize the path of Napoleon's army during the Russian campaign of 1812.

Each point on the map represents a geographical location through which the French army passed, with size indicating the number of surviving soldiers and color indicating the direction of movement (advance or retreat).

By interacting with the chart, the audience can easily view the direction of movement and have a comparison of advance versus retreat troops at each city.

In [ ]:
# Plot on Europe map
fig = px.scatter_mapbox(
    cities.dropna(),
    lat="lat",
    lon="long",
    color="direction",
    size="survivors",
    hover_name="city",
    hover_data={"survivors": True},
    zoom=4,
    center={"lat": 55, "lon": 30},
    mapbox_style="carto-positron",
    title="Napoleon's Troop Movements Across Europe "
)

fig.show()

The second map chart is also an interactive map that uses scatter_mapbox to visualize how the temperature affected the French troops and how many survivors were there at each city.

By interacting with the chart, the audience can see the fierceness of the temperature at the time when the French army passed through each city. The French army could not cope with the harsh winter, food shortages and disease, which resulted in the decline of survivors

In [ ]:
fig = px.scatter_mapbox(
    retreat,
    lat="lat",
    lon="long",
    color="temp",
    size="survivors",
    hover_name="city",
    hover_data={"survivors": True, "date": True, "temp": True, "city": True, "direction": True, "group": True},
    zoom=4,
    center={"lat": 55, "lon": 30},
    mapbox_style="carto-positron",
    title="Relationship between Survivors and Temperature on Map"
)

fig.show()

By using plotly.graph_objects library to visualize Napoleon's 1812 military campaign on a map of Europe. The third map recreates the advance and retreat of Napoleon's army during the Russian campaign, and shows the extreme temperatures during the retreat:


*   Advance direction: yellow line
*   Retreat direction: black line
*   Temperatures are shown below the retreat line, illustrating the extreme weather conditions when French troops retreating

By interacting with the map, the audience can have an overall view and comparisons: the size of each types of troops (advance and retreat) by cities, the harsh temperature during retreat movement.

As the scene unfolded, the audience gained a sharper perspective on the staggering defeat of Napoleon's army during their ill-fated Russia's campaign.

In [ ]:
# Create the base Mapbox figure
fig = go.Figure()

# Add Advance route (line weighted by survivors)
fig.add_trace(go.Scattermapbox(
    mode="lines+markers",
    lat=advance["lat"],
    lon=advance["long"],
    marker=dict(
        size=advance["survivors"] / 20000,   # scale markers by survivors
        color="#d62728"
    ),
    line=dict(
        width=4,              # must be one value
        color="#d62728"
    ),
    hovertext=[
    f"<b>{row['city']}</b><br>"
    f"Direction: Advance<br>"
    f"Survivors: {row['survivors']:,}<br>"
    f"Temp: {row['temp']}°C<br>"
    f"Date: {row['date'].strftime('%Y-%m-%d')}"
    for _, row in advance.iterrows()
    ],
    hoverinfo="text",
    name="Advance"
))

# Add Retreat route
fig.add_trace(go.Scattermapbox(
    mode="lines+markers",
    lat=retreat["lat"],
    lon=retreat["long"],
    marker=dict(size=6, color="#1f77b4"),
    line=dict(width=4, color="#1f77b4"),
    hovertext=[
        f"<b>{row['city']}</b><br>"
        f"Direction: Retreat<br>"
        f"Survivors: {row['survivors']:,}<br>"
        f"Temp: {row['temp']}°C<br>"
        f"Date: {row['date'].strftime('%Y-%m-%d')}"
        for _, row in retreat.iterrows()
    ],
    hoverinfo="text",
    name="Retreat"
))

# 8. Format map styling
fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        center={"lat": 55, "lon": 30},
        zoom=4
    ),
    title="🔥 Napoleon's Troop Movements Across Europe (Interactive)",
    title_x=0.5,
    margin=dict(l=0, r=0, t=50, b=0),
    legend=dict(title="Route", orientation="h", y=1.03)
)

fig.show()